In [ ]:
! pip install outscraper

In [ ]:
import json
from outscraper import ApiClient

def scrape_reviews(api_key, json_filename, output_json_filename):
    api_client = ApiClient(api_key=api_key)

    try:
        with open(json_filename, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

        all_reviews = {}

        for company_name, reviews_list in data.items():
            for reviews in reviews_list:
                for result in reviews:
                    place_id = result.get('place_id', '')

                    if place_id:
                        try:
                            reviews_data = api_client.google_maps_reviews(place_id, reviews_limit=3)
                            all_reviews[company_name] = {'place_id': place_id, 'reviews': reviews_data}
                        except Exception as e:
                            print(f"Error for {company_name}: {str(e)}")

        with open(output_json_filename, 'w', encoding='utf-8') as output_json_file:
            json.dump(all_reviews, output_json_file, ensure_ascii=False, indent=2)

        print(f"Reviews scraped and saved to '{output_json_filename}' successfully.")
    except Exception as e:
        print(f"Error: {str(e)}")

api_key = ' '
json_filename = 'output_results.json'
output_json_filename = 'reviews_output.json'

scrape_reviews(api_key, json_filename, output_json_filename)


In [ ]:
import json
import csv

#fileIn = "out_vpc.json"
#fileOut= "out_vpc.csv"

fileIn = "reviews_output.json"
fileOut= "reviews_output.csv"


#json_open = open(fileIn, 'r')
#json_load = json.load(json_open)

with open(fileIn, 'r', encoding='utf-8') as f:
    json_data = json.loads(f.read())


outDict = {}
glNo = 0

def nestLoop(el,argStr):
    global glNo
    headStr = ""
    nextStr = ""
    if type(el) is str: # 文字列
        nowNo = f'{glNo:08}'
        outElement = "{0}___{1}".format(nowNo,argStr)
        glNo += 1
        outDict.update( { outElement : el } )
    if type(el) is list: # リスト型
        for lp in el:
            nextStr = "{0}{1}".format(argStr , headStr)
            nestLoop( lp , nextStr)
            nextStr = ""
    if type(el) is dict: # 辞書型
        for lp in el.items():
            if argStr != "":
                headStr = "__"
            nextStr += "{0}{1}{2}".format( argStr , headStr , lp[0].strip() )
            nestLoop( el[ lp[0] ] , nextStr)
            nextStr = ""

nestLoop(json_data,"")

with open(r'{0}'.format(fileOut),'w',encoding='utf-8',newline="") as f:
    writer = csv.writer(f)
    for k, v in outDict.items():
        writer.writerow([k, v])


# no need to include the project below.

In [ ]:
import pandas as pd

df = pd.read_csv('reviews_output.csv')

# 一列目（最初の列）の中を2つのキーワードで検索
keyword1 = '__reviews__name'
keyword2 = '__reviews__place_id'

# 2つのキーワードのいずれかが含まれている行をフィルタリング
df_filtered = df[df.iloc[:, 0].str.contains(keyword1, case=False, na=False) | df.iloc[:, 0].str.contains(keyword2, case=False, na=False)]

df_filtered.to_csv('extract_out.csv', index=False)


In [ ]:
from io import StringIO
#df = pd.read_csv('extract_out.csv')

df2 = pd.read_csv(StringIO('extract_out.csv'))
print(df2)


In [ ]:
import pandas as pd

# CSVファイルのパスを指定して読み込み
csv_file_path = 'extract_out.csv'
df = pd.read_csv(csv_file_path)
print(df)

In [ ]:
#not working

# 'place_id' を含む列だけを残して他を削除
df_filtered = df[df.iloc[:, 0]].filter(like='place_id',axis=1)

df.head()

# Json to Pandas Data Frame

In [1]:
import pandas as pd

In [2]:
df = pd.read_json('reviews_output.json').transpose()

In [3]:
df.head()

,place_id,reviews
Activa,ChIJxbIbrsrzK4gRyKBqOoYlX6k,"[{'query': 'ChIJxbIbrsrzK4gRyKBqOoYlX6k', 'nam..."
Alma Mechanical,ChIJJ5_TX-IA1YkR6nZEdM6l0Sc,"[{'query': 'ChIJJ5_TX-IA1YkR6nZEdM6l0Sc', 'nam..."
CCR Build + Remodel Inc.,ChIJ-0bany_lLogRUBnpws5gzI4,"[{'query': 'ChIJ-0bany_lLogRUBnpws5gzI4', 'nam..."
Digreen Homes Inc.,ChIJ2yfX4tjU1IkRbct8G2DI2nA,"[{'query': 'ChIJ2yfX4tjU1IkRbct8G2DI2nA', 'nam..."
Ecocor Construction Ltd.,ChIJLeO7za44K4gRDb2YOuTLIO0,"[{'query': 'ChIJLeO7za44K4gRDb2YOuTLIO0', 'nam..."


In [4]:
df = df.rename_axis('name')

In [5]:
#df.columns.values[0] = 'name'

In [6]:
df.head()

,place_id,reviews
name,,
Activa,ChIJxbIbrsrzK4gRyKBqOoYlX6k,"[{'query': 'ChIJxbIbrsrzK4gRyKBqOoYlX6k', 'nam..."
Alma Mechanical,ChIJJ5_TX-IA1YkR6nZEdM6l0Sc,"[{'query': 'ChIJJ5_TX-IA1YkR6nZEdM6l0Sc', 'nam..."
CCR Build + Remodel Inc.,ChIJ-0bany_lLogRUBnpws5gzI4,"[{'query': 'ChIJ-0bany_lLogRUBnpws5gzI4', 'nam..."
Digreen Homes Inc.,ChIJ2yfX4tjU1IkRbct8G2DI2nA,"[{'query': 'ChIJ2yfX4tjU1IkRbct8G2DI2nA', 'nam..."
Ecocor Construction Ltd.,ChIJLeO7za44K4gRDb2YOuTLIO0,"[{'query': 'ChIJLeO7za44K4gRDb2YOuTLIO0', 'nam..."


In [7]:
df.reset_index(inplace=True)

# インデックスに 'name' という名前を付ける
df = df.rename(columns={'index': 'name'})

# 結果を表示
print(df)

                                   name                     place_id  \
0                                Activa  ChIJxbIbrsrzK4gRyKBqOoYlX6k   
1                       Alma Mechanical  ChIJJ5_TX-IA1YkR6nZEdM6l0Sc   
2              CCR Build + Remodel Inc.  ChIJ-0bany_lLogRUBnpws5gzI4   
3                    Digreen Homes Inc.  ChIJ2yfX4tjU1IkRbct8G2DI2nA   
4              Ecocor Construction Ltd.  ChIJLeO7za44K4gRDb2YOuTLIO0   
5                          Fusion Homes  ChIJE1rXfZiEK4gReqaygRXkfKU   
6                  Gemini Homebuilders   ChIJoQanqtiPK4gRlLrGQlyonZI   
7                            Great Gulf  ChIJk-aAJHXT1IkR162Fn2PZ_Xo   
8                       Greenpark Homes  ChIJ383bv6coK4gRR9vQBgnznpw   
9              Holman Construction Inc.  ChIJ00ISyOv7LogRxoLavOAzO9I   
10        Longwood Building Corporation  ChIJc3hN1S8QzkwR6Cw2aVsN90o   
11                  Menkes Developments  ChIJnxQrv2MtK4gRNJ398yKdSIQ   
12                Northbear Contracting  ChIJAQCEFZYv1okRZAqIqv9

In [8]:
new_df = df[['name', 'place_id']]

# 結果を表示
print(new_df)

                                   name                     place_id
0                                Activa  ChIJxbIbrsrzK4gRyKBqOoYlX6k
1                       Alma Mechanical  ChIJJ5_TX-IA1YkR6nZEdM6l0Sc
2              CCR Build + Remodel Inc.  ChIJ-0bany_lLogRUBnpws5gzI4
3                    Digreen Homes Inc.  ChIJ2yfX4tjU1IkRbct8G2DI2nA
4              Ecocor Construction Ltd.  ChIJLeO7za44K4gRDb2YOuTLIO0
5                          Fusion Homes  ChIJE1rXfZiEK4gReqaygRXkfKU
6                  Gemini Homebuilders   ChIJoQanqtiPK4gRlLrGQlyonZI
7                            Great Gulf  ChIJk-aAJHXT1IkR162Fn2PZ_Xo
8                       Greenpark Homes  ChIJ383bv6coK4gRR9vQBgnznpw
9              Holman Construction Inc.  ChIJ00ISyOv7LogRxoLavOAzO9I
10        Longwood Building Corporation  ChIJc3hN1S8QzkwR6Cw2aVsN90o
11                  Menkes Developments  ChIJnxQrv2MtK4gRNJ398yKdSIQ
12                Northbear Contracting  ChIJAQCEFZYv1okRZAqIqv96Ugc
13  Oke Woodsmith Building Systems